<a href="https://colab.research.google.com/github/HjrAmal/KNN-plants-leaves-type/blob/main/tomatoleaves_illness_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset.git

In [ ]:
!cd /content/PlantVillage-Dataset/raw/segmented/Tomato___healthy && ls

In [ ]:
import numpy as np

import cv2

from sklearn import neighbors



import glob

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier



In [ ]:
# image color RGB
healthy_path=glob.glob('/content/PlantVillage-Dataset/raw/color/Tomato___healthy/*.JPG')
late_path=glob.glob('/content/PlantVillage-Dataset/raw/color/Tomato___Late_blight/*.JPG')
mosaic_path=glob.glob('/content/PlantVillage-Dataset/raw/color/Tomato___Tomato_mosaic_virus/*.JPG')
curl_path=glob.glob('/content/PlantVillage-Dataset/raw/color/ Tomato___Tomato_Yellow_Leaf_Curl_Virus/*.JPG')
print(len(healthy_path)+len(late_path)+len(mosaic_path)+len(curl_path))




In [ ]:
# image gray GrayScale
healthy_path=glob.glob('/content/PlantVillage-Dataset/raw/grayscale/Tomato___healthy/*.JPG')
late_path=glob.glob('/content/PlantVillage-Dataset/raw/grayscale/Tomato___Late_blight/*.JPG')
mosaic_path=glob.glob('/content/PlantVillage-Dataset/raw/grayscale/Tomato___Tomato_mosaic_virus/*.JPG')
curl_path=glob.glob('/content/PlantVillage-Dataset/raw/grayscale/ Tomato___Tomato_Yellow_Leaf_Curl_Virus/*.JPG')
print(len(healthy_path)+len(late_path)+len(mosaic_path)+len(curl_path))

In [ ]:
# image segmented
healthy_path=glob.glob('/content/PlantVillage-Dataset/raw/segmented/Tomato___healthy/*.jpg')
late_path=glob.glob('/content/PlantVillage-Dataset/raw/segmented/Tomato___Late_blight/*.jpg')
mosaic_path=glob.glob('/content/PlantVillage-Dataset/raw/segmented/Tomato___Tomato_mosaic_virus/*.jpg')
curl_path=glob.glob('/content/PlantVillage-Dataset/raw/segmented/ Tomato___Tomato_Yellow_Leaf_Curl_Virus/*.jpg')
print(len(healthy_path)+len(late_path)+len(mosaic_path)+len(curl_path))






3873


In [ ]:
# Data Preparation

data = []
target = []
for img in healthy_path:
    img = cv2.imread(img)
    grayImg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    res = cv2.bitwise_and(img, img)
    data.append(res)
    target.append('healthy')



for img in late_path:
    img = cv2.imread(img)
    grayImg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    res = cv2.bitwise_and(img, img)
    data.append(res)
    target.append('Late blight')

for img in mosaic_path:
    img = cv2.imread(img)
    grayImg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  
    data.append(res)
    target.append('Tomato Mosaic Virus')  

for img in curl_path:
    img = cv2.imread(img)
    grayImg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    data.append(res)
    target.append('Tomato Yellow Leaf Curl Virus')  



In [ ]:
# Reshaping Data
data=np.reshape(data,(3873,196608))

In [ ]:
# Data Division
xtrain, xtest, ytrain, ytest = train_test_split(data, target, train_size=0.8,random_state=21)

In [ ]:
print(ytest)

In [ ]:
# Scale !!!!Important Accuracy 72.22% --> 77.77%
scaler = StandardScaler()
scaler.fit(xtrain)
xtrain = scaler.transform(xtrain)
xtest = scaler.transform(xtest)


In [ ]:
# PCA !!! important Accuracy 77,77% --> 80.55%
pca = PCA(0.95)
pca.fit(xtrain)
xtrain = pca.transform(xtrain)
xtest = pca.transform(xtest)

In [ ]:
print(np.shape(xtrain))

(3098, 1167)


In [ ]:
classifier = RandomForestClassifier(max_depth=10, random_state=0)
classifier.fit(xtrain, ytrain)

predicted = classifier.predict(xtest)
precision = classifier.score(xtest,ytest)
print(precision)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.84


In [ ]:
!pip install tpot

     |████████████████████████████████| 901kB 2.8MB/s 
     |████████████████████████████████| 153kB 65.3MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
  Created wheel for tpot: filename=TPOT-0.11.0-cp36-none-any.whl size=75684 sha256=a7573938d9c9296387243d05d4800854e7abf3f69f7b3168d4c028af0d97d4c6
  Stored in directory: /root/.cache/pip/wheels/5e/79/3b/49ccea9a29f28d0cdecbca22d71515c23bf45e1e65bc925cb8
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11955 sha256=44f0d8f86048953b225866ed050cf88ed900d30c9002180ff5e9da8f97a797b7
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built tpot stopit
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [ ]:
from tpot import TPOTClassifier

tpot = TPOTClassifier(generations=5)
tpot.fit(xtrain, ytrain)
prec=tpot.score(xtest, ytest)
print(prec)

In [ ]:
for k in range(1,15) :
  # Algorithm Training
  knn = neighbors.KNeighborsClassifier(n_neighbors=k,algorithm='auto',p=2)
  knn.fit(xtrain, ytrain)



  # Algorithm Test
  predicted = knn.predict(xtest)

  # Calcucating Algorithm Accuracy
  precision = knn.score(xtest, ytest)

  print('Precision: ', precision*100,'% pour k= ',k)

Precision:  82.3225806451613 % pour k=  1
Precision:  84.38709677419355 % pour k=  2
Precision:  80.7741935483871 % pour k=  3
Precision:  81.41935483870968 % pour k=  4
Precision:  81.03225806451613 % pour k=  5
Precision:  83.09677419354838 % pour k=  6
Precision:  81.41935483870968 % pour k=  7
Precision:  82.83870967741936 % pour k=  8
Precision:  81.29032258064515 % pour k=  9
Precision:  81.93548387096774 % pour k=  10
Precision:  81.16129032258065 % pour k=  11
Precision:  81.03225806451613 % pour k=  12
Precision:  80.90322580645162 % pour k=  13
Precision:  81.16129032258065 % pour k=  14
